In [31]:
import pandas as pd
import google.auth
from google.auth import compute_engine
from google.cloud import bigquery
from google.colab import auth
import google.cloud.bigquery.job as job
from datetime import datetime, timedelta
from collections import Counter
from  __builtin__ import any as b_any
from google.cloud import storage
from timeit import default_timer as timer

# Create the BQ client. This will ask you to log in the first time.

project_id = 'data-engineering-prod'
auth.authenticate_user()
client = bigquery.Client(project=project_id)

# list datasets and number of views/tables
print("Collecting data on all views in " + project_id + ". This may take some time...")
views = []
# orion, orex, payment, identity, comms, flows, tardis, he, bast, smint, past, jaws
orion_datasets = ["orion","orex","payment","identity","comms","flows","sre",
                  "tardis","bast","smint","payments","jaws","onboarding","pace"]
dataset_count = 0
for dataset in client.list_datasets():
  if dataset.dataset_id.startswith('u_'): continue
  if not any(name in dataset.dataset_id for name in orion_datasets): continue
  dataset_views = list(client.list_tables(dataset=dataset.reference))
  dataset_count += 1
  for view in dataset_views:
      views.append(view)

  print(dataset.dataset_id + " contains {:d} views".format(len(dataset_views)))

print("\n" + project_id + " contains {:d} dataset".format(dataset_count))
print("\nDone!")

In [32]:
# list datasets and number of views/tables
print("Collecting data on all views in " + project_id + ". This may take some time...")
views = []
dataset_count = 0
dataset_info = []
for dataset in client.list_datasets():
  if dataset.dataset_id.startswith('u_'): continue
  dataset_views = list(client.list_tables(dataset=dataset.reference))
  dataset_info.append([dataset.dataset_id, len(dataset_views)])

  dataset_count += 1
  for view in dataset_views:
  #  try:
  #    dataset_ref = bigquery.DatasetReference(project_id, dataset.dataset_id)
  #    view_ref = dataset_ref.table(view.table_id)
  #    v = client.get_table(view_ref)
      views.append(view)

  #  except Exception, err:
  #    print(err)
  #    pass

  print(dataset.dataset_id + " contains {:d} views".format(len(dataset_views)))

print("\n" + project_id + " contains {:d} dataset".format(dataset_count))
print("\nDone!")


acquisitions_reporting contains 2 views

bast_reporting contains 2 views

brunel_billing_exceptions contains 0 views

brunel_orion_analytics_cloud_composer_test contains 30 views

cais contains 3 views

compliance_reporting contains 10 views

data_analytics_reference contains 1 views

dse_reporting contains 4 views

eligibility contains 147 views

home_moves_reporting contains 2 views

migration_reporting contains 2 views

monitoring contains 2 views

orion_analytics_scratch_space contains 0 views

orion_finance contains 5 views

orion_reporting contains 18 views

pace_reporting contains 4 views

sse_eligibility contains 16 views

switching_overcharge contains 8 views

test_log_export contains 1 views

test_log_export_eu contains 1 views

usabilla contains 23 views

andromeda-data-nonprod contains 21 dataset

Done!


In [33]:
from google.cloud import storage

# UPLOAD VIEWS 
# write data to newline delimited json file
json_file = "data_engineering_prod_bq_analysis.json"
columns = ["full_table_id","dataset_id","type","created"] #"query","location","multi_regional","bytes","created", "last_modified"]
df = pd.DataFrame(data, columns=columns)
# bq requires newline delimited json so append line break
file = open(json_file, "w")
for row in df.iterrows():
  row[1].to_json(file)
  file.write("\n")
file.close()

# write the results to a gc bucket 
client = storage.Client(project=project_id)
bucket = client.get_bucket("data-engineering-prod-bq-analysis")
blob = bucket.blob(json_file)

with open('data_engineering_prod_bq_analysis.json', 'rb') as file:
  blob.upload_from_file(file)

# write data to data-integration-prod
# to update the results we will wipe the existing table and replace it
project_id = 'data-integration-prod'
dataset_id = 'data_engineering_prod_bq_analysis'
table_id = 'data_engineering_prod_bq_analysis'
full_table_id = project_id + '.' + dataset_id + '.' + table_id
client = bigquery.Client(project=project_id)

# remove existing table
client.delete_table(full_table_id, not_found_ok=True) 

# write new table
job_config = bigquery.LoadJobConfig(autodetect=True, 
            source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON)
uri = "gs://data-engineering-prod-bq-analysis/data_engineering_prod_bq_analysis.json"
load_job = client.load_table_from_uri(
    uri, full_table_id, job_config=job_config)  # Make an API request.
load_job.result() 

# check the number of rows loaded into the table is correct
destination_table = client.get_table(full_table_id)
print("Loaded {} rows.".format(destination_table.num_rows))
assert(len(views) == destination_table.num_rows)

In [36]:
from google.cloud import storage

# UPLOAD DATASET INFO 
# write data to newline delimited json file
json_file = "data_engineering_prod_bq_dataset_analysis.json"
columns = ["dataset_id","number_of_objects"] 
df = pd.DataFrame(dataset_info, columns=columns)
# bq requires newline delimited json so append line break
file = open(json_file, "w")
for row in df.iterrows():
  row[1].to_json(file)
  file.write("\n")
file.close()

# write the results to a gc bucket 
client = storage.Client(project=project_id)
bucket = client.get_bucket("data-engineering-prod-bq-analysis")
blob = bucket.blob(json_file)

with open('data_engineering_prod_bq_dataset_analysis.json', 'rb') as file:
  blob.upload_from_file(file)

# write data to data-integration-prod
# to update the results we will wipe the existing table and replace it
project_id = 'data-integration-prod'
dataset_id = 'data_engineering_prod_bq_analysis'
table_id = 'data_engineering_prod_bq_dataset_analysis'
full_table_id = project_id + '.' + dataset_id + '.' + table_id
client = bigquery.Client(project=project_id)

# remove existing table
client.delete_table(full_table_id, not_found_ok=True) 

# write new table
job_config = bigquery.LoadJobConfig(autodetect=True, 
            source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON)
uri = "gs://data-engineering-prod-bq-analysis/data_engineering_prod_bq_dataset_analysis.json"
load_job = client.load_table_from_uri(
    uri, full_table_id, job_config=job_config)  # Make an API request.
load_job.result() 

# check the number of rows loaded into the table is correct
destination_table = client.get_table(full_table_id)
print("Loaded {} rows.".format(destination_table.num_rows))
assert(len(dataset_info) == destination_table.num_rows)

Loaded 281 rows.


In [37]:
# VIEW/TABLE DEPENDENCIES

# check which views use which tables
views_only = filter(lambda view: view.table_type == "VIEW", views)
tables_only = filter(lambda table: table.table_type == "TABLE", views)
print("Table number: {}".format(len(tables_only)))

view_table_dependencies = []
for table in tables_only:
  views_list = []
  for view in views_only:
    if table.table_id in view.view_query:
      views_list.append(view.table_id)
  view_table_dependencies.append([table.full_table_id, views_list])

Table number: 118


In [38]:
# write data to newline delimited json file
json_file = "data_engineering_prod_bq_table_view_dependency_analysis.json"
columns = ["full_table_id","views_using_table"]
df = pd.DataFrame(view_table_dependencies, columns=columns)
# bq requires newline delimited json so append line break
file = open(json_file, "w")
for row in df.iterrows():
  row[1].to_json(file)
  file.write("\n")
file.close()

In [39]:
from google.cloud import storage

# write the results to a gc bucket 
client = storage.Client(project=project_id)
bucket = client.get_bucket("data-engineering-prod-bq-analysis")
blob = bucket.blob(json_file)

with open('data_engineering_prod_bq_table_view_dependency_analysis.json', 'rb') as file:
  blob.upload_from_file(file)

In [40]:
# write data to data-integration-prod
# to update the results we will wipe the existing table and replace it
project_id = 'data-integration-prod'
dataset_id = 'data_engineering_prod_bq_analysis'
table_id = 'data_engineering_prod_bq_table_view_dependency_analysis'
full_table_id = project_id + '.' + dataset_id + '.' + table_id
client = bigquery.Client(project=project_id)

# remove existing table
client.delete_table(full_table_id, not_found_ok=True)

# write new table
job_config = bigquery.LoadJobConfig(autodetect=True, 
            source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON)
uri = "gs://data-engineering-prod-bq-analysis/data_engineering_prod_bq_table_view_dependency_analysis.json"
load_job = client.load_table_from_uri(
    uri, full_table_id, job_config=job_config)  # Make an API request.
load_job.result() 

# check the number of rows loaded into the table is correct
destination_table = client.get_table(full_table_id)
print("Loaded {} rows.".format(destination_table.num_rows))
assert(len(view_table_dependencies) == destination_table.num_rows)

Loaded 118 rows.


In [41]:
### COST ANALYSIS ###
# This section is for the cost analysis of existing tables and views in the project

# https://cloud.google.com/bigquery/pricing

# - cost of storage (bytes)
# - region/multi
# - partitioned
# - expiration

# collect relevant data
cost_data = []

# https://cloud.google.com/bigquery/docs/locations
# indicates "EU" & "US" are multi-regional locations
multi_regional = ["EU", "US"]

for view in views:
  multi = True if view.location in multi_regional else False

  d = [view.full_table_id,
      view.location, 
      multi,
      view.num_bytes, 
      view.expires.strftime('%Y-%m-%d %H:%M:%S') if view.expires != None else None,
      view.partitioning_type,
      view.partition_expiration.strftime('%Y-%m-%d %H:%M:%S') if view.partition_expiration != None else None,
      ]
  cost_data.append(d)

In [42]:
# write data to newline delimited json file
json_file = "data_engineering_prod_bq_cost_analysis.json"
columns = ["full_table_id","location","multi","bytes","expiration","partition","partition_expiration"]
df = pd.DataFrame(cost_data, columns=columns)
# bq requires newline delimited json so append line break
file = open(json_file, "w")
for row in df.iterrows():
  row[1].to_json(file)
  file.write("\n")
file.close()

In [43]:
from google.cloud import storage

# write the results to a gc bucket 
client = storage.Client(project=project_id)
bucket = client.get_bucket("data-engineering-prod-bq-analysis")
blob = bucket.blob(json_file)

with open('data_engineering_prod_bq_cost_analysis.json', 'rb') as file:
  blob.upload_from_file(file)

In [44]:
# write data to data-integration-prod
# to update the results we will wipe the existing table and replace it
project_id = 'data-integration-prod'
dataset_id = 'data_engineering_prod_bq_analysis'
table_id = 'data_engineering_prod_bq_cost_analysis'
full_table_id = project_id + '.' + dataset_id + '.' + table_id
client = bigquery.Client(project=project_id)

# remove existing table
client.delete_table(full_table_id, not_found_ok=True)

# write new table
job_config = bigquery.LoadJobConfig(autodetect=True, 
            source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON)
uri = "gs://data-engineering-prod-bq-analysis/data_engineering_prod_bq_cost_analysis.json"
load_job = client.load_table_from_uri(
    uri, full_table_id, job_config=job_config)  # Make an API request.
load_job.result() 

# check the number of rows loaded into the table is correct
destination_table = client.get_table(full_table_id)
print("Loaded {} rows.".format(destination_table.num_rows))
assert(len(cost_data) == destination_table.num_rows)

Loaded 281 rows.
